In [18]:
import openai
import yfinance as yf
import json


def get_stock_price(symbol):
    stock = yf.Ticker(symbol)
    price = stock.info["currentPrice"]
    return price


def get_latest_company_news(symbol):
    stock = yf.Ticker(symbol)
    news = stock.news

    news_list = []
    num = 1
    for item in news[:3]:
        news_list.append(
            f"{num}: title : {item['title']}, publisher: {item['publisher']}, link : {item['link']}"
        )
        num += 1
    return news_list


tools_list = [
    {
        "type": "function",
        "function": {
            "name": "get_stock_price",
            "description": "Get the current stock price for a given symbol",
            "parameters": {
                "type": "object",
                "properties": {
                    "symbol": {
                        "type": "string",
                        "description": "The stock symbol to get the price for",
                    }
                },
                "required": ["symbol"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_latest_company_news",
            "description": "Get the latest news for a given company symbol",
            "parameters": {
                "type": "object",
                "properties": {
                    "symbol": {
                        "type": "string",
                        "description": "The stock symbol to get the latest news for",
                    }
                },
                "required": ["symbol"],
            },
        },
    },
]

In [19]:
import os


client = openai.OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [20]:
instruction="""
너는 주식 분석 전문가야

규칙:
1. 사용자가 회사 또는 종목에 대해 물어보면, function을 활용해서 회사의 주식 가격과 최신 뉴스를 답해줘.
2. 모든 답변은 한국어로 해줘.
"""

assistant = client.beta.assistants.create(
    name="Company Stock Analyst",
    instructions=instruction,
    model='gpt-4o-mini',
    tools = tools_list,
)

In [21]:
thread = client.beta.threads.create()


In [22]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="테슬라의 최근 주가와 최신 뉴스에 대해 알려줘"
)

In [23]:
run = client.beta.threads.runs.create(thread_id=thread.id, assistant_id=assistant.id)

In [24]:
import time


while run.status not in ['completed', 'failed', "requires_action"]:
    run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)
    print(run.status)
    time.sleep(1)

print(run.status)

requires_action
requires_action


In [25]:
tools_to_call = run.required_action.submit_tool_outputs.tool_calls
print('필요 호출 개수: ', len(tools_to_call))
print(tools_to_call)

필요 호출 개수:  2
[RequiredActionFunctionToolCall(id='call_dMVCAvoEGsvALIEnqRMS7hiH', function=Function(arguments='{"symbol": "TSLA"}', name='get_stock_price'), type='function'), RequiredActionFunctionToolCall(id='call_edvegnwB8zuYpGp3b79ptwmy', function=Function(arguments='{"symbol": "TSLA"}', name='get_latest_company_news'), type='function')]


In [26]:
print(tools_to_call[0].function.name)
print(tools_to_call[0].function.arguments)
print(tools_to_call[1].function.name)
print(tools_to_call[1].function.arguments)

get_stock_price
{"symbol": "TSLA"}
get_latest_company_news
{"symbol": "TSLA"}


In [27]:
tools_output_array = []
for each_tool in tools_to_call:
    tool_call_id = each_tool.id
    function_name = each_tool.function.name
    function_arg = each_tool.function.arguments
    function_arg = json.loads(function_arg)

    if function_name == "get_stock_price":
        output = get_stock_price(function_arg["symbol"])
    if function_name == "get_latest_company_news":
        output = get_latest_company_news(function_arg["symbol"])

    tools_output_array.append({"tool_call_id": tool_call_id, "output": json.dumps(output)})

print(tools_output_array)

[{'tool_call_id': 'call_dMVCAvoEGsvALIEnqRMS7hiH', 'output': '338.23'}, {'tool_call_id': 'call_edvegnwB8zuYpGp3b79ptwmy', 'output': '["1: title : Inside Elon Musk\\u2019s Quest to Beat OpenAI at Its Own Game, publisher: The Wall Street Journal, link : https://finance.yahoo.com/m/1bff3d65-da6f-3891-a827-330619025891/inside-elon-musk%E2%80%99s-quest-to.html", "2: title : Trump is set to end the EV tax credit. Here are 7 electric cars to buy before it goes away, publisher: Quartz, link : https://finance.yahoo.com/m/be8c2e45-2b0f-3f5d-986d-0385fdc1fb42/trump-is-set-to-end-the-ev.html", "3: title : Lidar maker Hesai sees device becoming as common as airbags, publisher: Bloomberg, link : https://finance.yahoo.com/news/lidar-maker-hesai-sees-technology-023504385.html"]'}]


In [28]:
run = client.beta.threads.runs.submit_tool_outputs(thread_id=thread.id, run_id=run.id, tool_outputs=tools_output_array)

In [29]:
messages = client.beta.threads.messages.list(thread_id=thread.id)

In [33]:
for each in messages:
    print(each.role + ": " + each.content[0].text.value)
    print("================")

assistant: 현재 테슬라(TSLA)의 주가는 **$338.23**입니다.

최신 뉴스들은 다음과 같습니다:

1. **[Inside Elon Musk’s Quest to Beat OpenAI at Its Own Game](https://finance.yahoo.com/m/1bff3d65-da6f-3891-a827-330619025891/inside-elon-musk%E2%80%99s-quest-to.html)** - 출처: The Wall Street Journal
2. **[Trump is set to end the EV tax credit. Here are 7 electric cars to buy before it goes away](https://finance.yahoo.com/m/be8c2e45-2b0f-3f5d-986d-0385fdc1fb42/trump-is-set-to-end-the-ev.html)** - 출처: Quartz
3. **[Lidar maker Hesai sees device becoming as common as airbags](https://finance.yahoo.com/news/lidar-maker-hesai-sees-technology-023504385.html)** - 출처: Bloomberg

더 궁금한 점이 있으면 말씀해 주세요!
user: 테슬라의 최근 주가와 최신 뉴스에 대해 알려줘
